In [173]:
# import essential libraries
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [174]:

# read in csv 
df = pd.read_csv('training_11_features.csv')
df = df.dropna()
# df = df[df['ritmi'] != 0]
df = df.reset_index(drop=True)
print(df.shape)
df.head()

(1803, 11)


,ritmi,age,sex,height,weight,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,2,29.0,1,164.0,56.0,0,0.0,0,1,0,1
1,0,59.0,0,156.0,75.0,0,0.0,0,1,0,9
2,2,84.0,1,152.0,51.0,0,0.0,0,1,0,7
3,0,79.0,0,172.0,66.0,0,0.0,0,1,0,5
4,1,67.0,0,178.0,73.0,4,0.0,0,1,0,5


In [175]:
# convert all columns' types to int64
df['age'] = df['age'].astype('int64')
df['height'] = df['height'].astype('int64')
df['weight'] = df['weight'].astype('int64')
df['validated_by'] = df['validated_by'].astype('int64')

# get info for columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803 entries, 0 to 1802
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   ritmi               1803 non-null   int64
 1   age                 1803 non-null   int64
 2   sex                 1803 non-null   int64
 3   height              1803 non-null   int64
 4   weight              1803 non-null   int64
 5   heart_axis          1803 non-null   int64
 6   validated_by        1803 non-null   int64
 7   second_opinion      1803 non-null   int64
 8   validated_by_human  1803 non-null   int64
 9   pacemaker           1803 non-null   int64
 10  strat_fold          1803 non-null   int64
dtypes: int64(11)
memory usage: 155.1 KB


In [176]:
# train-test split
X = df.drop(columns='ritmi')
y = df['ritmi']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size = 0.25, random_state = 246)

In [177]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (1352, 10)
y_train (1352,)
X_test (451, 10)
y_test (451,)


In [178]:
#Random Forest

In [195]:
# Plug in appropriate max_depth and random_state parameters
rf = RandomForestClassifier()
rf_param_grid = {'n_estimators': [600], 'criterion': ['entropy'], 'max_depth': [60]} #0.4615443314230772

rf_cv= GridSearchCV(rf,rf_param_grid,cv=7,n_jobs=-1)

rf_cv.fit(X_train,y_train)

print("Best Score:" + str(rf_cv.best_score_))
print("Best Parameters: " + str(rf_cv.best_params_))

X_train (1352, 10)
y_train (1352,)
Best Score:0.46006394652300314
Best Parameters: {'criterion': 'entropy', 'max_depth': 60, 'n_estimators': 600}


In [180]:
y_pred = rf_cv.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.45      0.46       162
           1       0.43      0.40      0.41       117
           2       0.48      0.52      0.50       172

    accuracy                           0.46       451
   macro avg       0.46      0.46      0.46       451
weighted avg       0.46      0.46      0.46       451



In [181]:
# K Neighbors Classifier

In [182]:
clfl2 = KNeighborsClassifier()
parameters = {'n_neighbors': [150], 'weights': ['distance'], 'metric': ['euclidean']} #0.4889326226595599

fitmodel = GridSearchCV(clfl2, param_grid=parameters, cv=5, refit=True, scoring="accuracy", n_jobs=-1, verbose=3)
fitmodel.fit(X_train, y_train)
print(fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
KNeighborsClassifier(metric='euclidean', n_neighbors=150, weights='distance') {'metric': 'euclidean', 'n_neighbors': 150, 'weights': 'distance'} 0.4881918819188192


In [183]:
y_pred = fitmodel.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.40      0.43       162
           1       0.49      0.47      0.48       117
           2       0.43      0.50      0.46       172

    accuracy                           0.46       451
   macro avg       0.46      0.46      0.46       451
weighted avg       0.46      0.46      0.46       451



In [184]:
#Convo1D
!pip install  --default-timeout=100 tensorflow

In [219]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from tensorflow.keras import layers

In [220]:
n_timesteps=5
n_features=1
model=Sequential()
model.add(layers.Conv1D(64,kernel_size=2,activation='relu',input_shape=(n_timesteps,n_features)))

model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

In [221]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [222]:
n_steps=5
X, y = split_sequence(df.ritmi, n_steps)
print(X)
print(y)
X = X.reshape((X.shape[0], X.shape[1], n_features))

[[2 0 2 0 1]
 [0 2 0 1 1]
 [2 0 1 1 1]
 ...
 [2 1 1 0 2]
 [1 1 0 2 2]
 [1 0 2 2 2]]
[1 1 2 ... 2 2 1]


In [223]:
model.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [224]:
history=model.fit(X,y, epochs=15, batch_size=64, verbose=1)

Epoch 1/15
29/29 [==============================] - 0s 1ms/step - loss: 1.0218 - acc: 0.2736
Epoch 2/15
29/29 [==============================] - 0s 2ms/step - loss: 0.7853 - acc: 0.2647
Epoch 3/15
29/29 [==============================] - 0s 1ms/step - loss: 0.7751 - acc: 0.2631
Epoch 4/15
29/29 [==============================] - 0s 1ms/step - loss: 0.7677 - acc: 0.2631
Epoch 5/15
29/29 [==============================] - 0s 1ms/step - loss: 0.7624 - acc: 0.2631
Epoch 6/15
29/29 [==============================] - 0s 2ms/step - loss: 0.7578 - acc: 0.2625
Epoch 7/15
29/29 [==============================] - 0s 1ms/step - loss: 0.7516 - acc: 0.2625
Epoch 8/15
29/29 [==============================] - 0s 1ms/step - loss: 0.7491 - acc: 0.2636
Epoch 9/15
29/29 [==============================] - 0s 1ms/step - loss: 0.7469 - acc: 0.2631
Epoch 10/15
29/29 [==============================] - 0s 2ms/step - loss: 0.7404 - acc: 0.2625
Epoch 11/15
29/29 [==============================] - 0s 1ms/step - lo

In [228]:
y_preds = model.predict(X)

57/57 [==============================] - 0s 684us/step


In [232]:
print(preds)

[[1.1678572 ]
 [1.6459259 ]
 [1.4074236 ]
 ...
 [0.77068365]
 [1.0914671 ]
 [1.0888737 ]]


In [234]:
print(classification_report(y_preds))

TypeError: classification_report() missing 1 required positional argument: 'y_pred'